In [21]:
import gspread
from google.oauth2.service_account import Credentials
import re
import configparser
import pandas as pd
import uuid
import sqlite3

In [ ]:
# return list of source sheets
def source_sheets(contact_config):
    sheet_info = []
    for key in contact_config.keys():
        if 'source_sheet' in key:
            sheet, column, row = contact_config.get(key).split(',')
            sheet_info.append( (sheet, int(column), int(row)) )
    return tuple(sheet_info)

# split emails if they put two in a cell
def split_emails(raw_list):
    split_result = []
    for entry in raw_list:
        # Split on anything that isn't a valid email character
        parts = re.split(r'[^a-zA-Z0-9._%+\-@]+', entry)
        split_result.extend([p.strip() for p in parts if p.strip()])
    return split_result

# Basic email pattern: username@domain
EMAIL_REGEX = re.compile(r"^[a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+$")
def is_valid_email(email):
    return EMAIL_REGEX.match(email) is not None

# builds a vcard
def generate_vcard(first, last, fn, mobile, email, notes):
    return f"""BEGIN:VCARD
VERSION:3.0
N:{last};{first};;;
FN:{fn}
TEL;TYPE=CELL:{mobile}
EMAIL:{email}
NOTE:{notes}
END:VCARD
"""

In [23]:
# get config
cp = configparser.ConfigParser() 
cp.read('config.ini')
config = dict(cp['contacts'])

# Setup auth and client
SCOPES = ['https://www.googleapis.com/auth/drive']
creds = Credentials.from_service_account_file(config.get('credentials'), scopes=SCOPES)
client = gspread.authorize(creds)

In [24]:
# Open your spreadsheet by name or URL
spreadsheet = client.open(config.get('workbook'))
records = {}

In [25]:
data = spreadsheet.worksheet("AGIMBA Contacts").get_values("A1:E")
# Convert to list of dicts
keys = data[0]
rows = data[1:]

for contact in [dict(zip(keys, row)) for row in rows]:
    email = contact.get('email').lower()
    if not is_valid_email(email):
        continue
    records[email] = {'email': email,
                                'first_name': contact.get('first_name'),
                                'last_name': contact.get('last_name'),
                                'formatted_name': f"{contact.get('first_name')} {contact.get('last_name')}",
                                'phone': contact.get('phone'),
                                'notes': '',
                               }

In [26]:
data = spreadsheet.worksheet("AGMB Roster").get_values("A1:F")
# Convert to list of dicts
keys = ['student_first_name', 'student_last_name', 'grade', 'section', 'parent_name', 'email']
rows = data[1:]

for contact in [dict(zip(keys, row)) for row in rows]:
    raw_email = contact.get('email').lower()
    for email in split_emails([raw_email,]):
        if not is_valid_email(email):
            continue
        if not records.get(email):
            records[email] = {'email': email,
                                        'first_name': contact.get('parent_name'),
                                        'last_name': '',
                                        'formatted_name': contact.get('parent_name'),
                                        'phone': '',
                                        'notes': '',
                                       }    
        records[email]['notes'] = '\n'.join([ f"{records.get(email).get('notes')}",
             f"Student: {contact.get('student_first_name').strip()} {contact.get('student_last_name').strip()} -> {contact.get('section').strip()}"] )

In [27]:
data = spreadsheet.worksheet("AGIMBA Org").get_values("A1:B")
# Convert to list of dicts
keys = ['role', 'email']
rows = data[1:]

for contact in [dict(zip(keys, row)) for row in rows]:
    for email in split_emails([contact.get('email').lower(),]):
        if not is_valid_email(email):
            continue
        if not records.get(email):
            records[email] = {'email': email,
                                        'first_name': contact.get('role'),
                                        'last_name': '',
                                        'formatted_name': contact.get('role'),
                                        'phone': '',
                                        'notes': '',
                                       }

In [28]:
data = spreadsheet.worksheet("AG Staff").get_values("A1:C")
# Convert to list of dicts
keys = ['first_name', 'last_name', 'email']
rows = data[1:]

for contact in [dict(zip(keys, row)) for row in rows]:
    for email in split_emails([contact.get('email').lower(),]):
        if not is_valid_email(email):
            continue
        if not records.get(email):
            records[email] = {'email': email,
                                        'first_name': contact.get('first_name'),
                                        'last_name': contact.get('last_name'),
                                        'formatted_name': f"{contact.get('first_name')} {contact.get('last_name')}",
                                        'phone': '',
                                        'notes': '',
                                       }

In [29]:
with open('contacts.vcf', 'w') as vcf_file:
    for key in sorted(records):
        vcard = generate_vcard(
            records.get(key).get('first_name'),
            records.get(key).get('last_name'),
            records.get(key).get('formatted_name'),
            records.get(key).get('phone'),
            records.get(key).get('email'),
            records.get(key).get('notes')
        )
        vcf_file.write(vcard + '\n')
        #print(vcard)

In [30]:
# Build list of lists as need for sheet update
update_list = [ [email,] for email in sorted(records.keys()) ]

# Clear old data
sheet_email_lists = spreadsheet.worksheet(config.get('email_sheet'))
sheet_email_lists.batch_clear([config.get('email_range')])

# Prepare for writing
sheet_email_lists.update(range_name=f'{config.get('email_range')}{len(update_list)+1}', values=update_list )

print(f"Wrote {len(update_list)} unique emails to '{sheet_email_lists.title}' sheet.")

Wrote 110 unique emails to 'EmailLists' sheet.


In [11]:
wanted_keys = ["first_name", "last_name", "email", "phone"]

result = [
    {k: v for k, v in inner.items() if k in wanted_keys}
    for inner in records.values()
]

pd.DataFrame(result).to_csv("agimba_volunteers.csv", index=False)

In [12]:
contact_list = [records.get(contact) for contact in records]

In [13]:
data = spreadsheet.worksheet("AGMB Roster").get_values("A1:F")
# Convert to list of dicts
keys = ['student_first_name', 'student_last_name', 'grade', 'section', 'parent_name', 'email']
rows = data[1:]
roster = [dict(zip(keys, row)) for row in rows]

In [14]:
df = pd.DataFrame(roster)
df['grade'] = pd.to_numeric(df['grade'], errors='coerce')
df['grad_year'] = 2026 + (12 - df['grade'])
df['activity'] = 'Marching Band'
df['first_name'] = df['student_first_name']
df['last_name'] = df['student_last_name']
df['parent_email'] = df['email']
df['activity_year'] = 2025
df['student_id'] = [str(uuid.uuid4()) for _ in range(len(df))]


# Connect to SQLite file
with sqlite3.connect('agimba.db') as conn:
    # student table
    columns_to_save = ['first_name', 'last_name', 'grad_year', 'student_id']
    df[columns_to_save].to_sql('students', conn, if_exists='replace', index=False)

    # Save to parents table
    columns_to_save = ['parent_name', 'parent_email']
    df_parents = df[columns_to_save].drop_duplicates()
    df_parents['parent_id'] = [str(uuid.uuid4()) for _ in range(len(df_parents))]
    df_parents.to_sql('parents', conn, if_exists='replace', index=False)

    # Save to activities table
    columns_to_save = ['student_id', 'activity_year', 'activity', 'section']
    df_activities = df[columns_to_save].drop_duplicates()
    df_activities.to_sql('activities', conn, if_exists='replace', index=False)

    # save to student_partent_mapping
    df_mapping = df.merge(df_parents[['parent_email', 'parent_id']], on="parent_email", how="left")
    columns_to_save = ['student_id', 'parent_id']
    df_mapping = df_mapping[columns_to_save].drop_duplicates()
    df_mapping.to_sql('student_parent_mapping', conn, if_exists='replace', index=False)

In [15]:
df_mapping

,student_id,parent_id
0,c7588c42-acf1-495c-ad6c-3ad0afc63183,7246d4b2-f6a3-4167-8f01-1407960d9142
1,0bcb096f-48fd-4a25-915a-30ef831b7b5a,7004f7a5-a874-4508-977c-8421bb340a25
2,c44da4ff-7257-40c1-9888-a5f8ff58fc53,e3f92b68-6746-4001-873f-a4a47328088d
3,ceae8204-5106-4811-a298-a2d07dbe2f84,7b9f4eef-2de1-4721-8dbb-e2dd2422007c
4,aaee5dce-562a-415d-96ce-4552f8c3b14f,82822d40-339b-400e-ae57-86066f299c65
...,...,...
71,022677e3-cf1a-41ac-bdc4-64033385fec7,0009349f-2d7d-4217-a2f8-7de813a0b979
72,59c37b03-a464-482a-bf06-20b0d097c4ab,4dbd2263-af0a-43fa-a4cc-9df8d92a8ff5
73,a19ff2dd-6beb-4e73-b330-d1a135eaa2ea,26efffde-a572-43e8-a06a-76de55281fa9
74,52db4422-1bd3-49f4-b12a-687130e5e534,379d0ee0-120e-4b2a-942f-26ae04148cb8
